# AI Brochure Generator


## Project Overview

In this project, we built a highly efficient brouchure generator using LLM. We also added the option of translating the generated brochure. The solution process can be divided into three main stages. To build our graphical interface we used gradio wich is an open-source Python framework that simplifies the creation of interactive web interfaces for machine learning models, APIs, or any Python function

* The first step is to build a website scrapper that can retrieve the content of a given url website.
* After scrapping the website, we'll send the useful website content to an LLM model. The LLM model will generate a brochure by summarizing and extracting the useful information. We choose Chatgpt and Claude to do this task.
* The final step is to send the generated brochure to another LLM for translation into the desired language.In our context, we decided to translate it into French.
  


In [1]:
#Import all required libraries for our project
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import gradio as gr
import anthropic

In [2]:
# Check that our all the LLM API Keys are found

load_dotenv(override=True)
load_dotenv('../.env.anthropic', override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
#google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [3]:
Model = "gpt-4o-mini"
openai = OpenAI()

claude = anthropic.Anthropic()
claude_model = "claude-3-haiku-20240307"

In [4]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
# This a small web Scrapper that scrap only static wbesite base on the url
class ScrapStaticWebsite:
    def __init__(self,url,headers):
        self.url = url
        self.headers = headers
        response = requests.get(self.url, headers=self.headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
        
    def get_site_contents(self):
         return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
           
                

In [6]:
site = ScrapStaticWebsite("https://www.jrvs.ca/",headers)

In [7]:
print(site.get_site_contents())

Webpage Title:
Jarvis Consulting Group | Innovate. Empower. Impact
Webpage Contents:
EMPOWERING AI, DATA & TECH
INNOVATION FOR POSITIVE IMPACT
Our Expertise
MirageAI
DATA MODERNIZATION WITHOUT COMPROMISING SECURITY
Learn more
EMPOWERING AI, DATA & TECH
INNOVATION FOR POSITIVE IMPACT
Our Expertise
MirageAI
DATA MODERNIZATION WITHOUT COMPROMISING SECURITY
Learn more
EMPOWERING AI, DATA & TECH
INNOVATION FOR POSITIVE IMPACT
Our Expertise
MirageAI
DATA MODERNIZATION WITHOUT COMPROMISING SECURITY
Learn more
EMPOWERING AI, DATA & TECH
INNOVATION FOR POSITIVE IMPACT
Our Expertise
MirageAI
DATA MODERNIZATION WITHOUT COMPROMISING SECURITY
Learn more
MirageAI
Services
Company
Careers
Select Language
English
French (Canada)
At Jarvis,
WE EMPOWER INNOVATION
IN DATA, AI & TECHNOLOGY
TO DRIVE POSITIVE IMPACT
Our Expertise
MirageAI
DATA MODERNIZATION WITHOUT COMPROMISING SECURITY
Learn more
At Jarvis,
WE EMPOWER INNOVATION
IN DATA, AI & TECHNOLOGY
TO DRIVE POSITIVE IMPACT
Our Expertise
MirageAI
DATA 

In [8]:
print(site.links)

['./#ai', './#ai', './#ai', './#ai', './story/mirageai-advantage#background', './#ai', './#ai', './#ai', './#ai', './story/mirageai-advantage#background', './#ai', './#ai', './#ai', './#ai', './story/mirageai-advantage#background', './#ai', './#ai', './#ai', './#ai', './story/mirageai-advantage#background', './', './mirage-ai', './about-us', './careers', './', './#ai', './story/mirageai-advantage#background', './#ai', './story/mirageai-advantage#background', './#ai', './story/mirageai-advantage#background', './#ai', './story/mirageai-advantage#background', './#ai', './#advanced', './#cloud', './#digital', './#financial', './#talent', './data-and-ai', './data-and-ai', './data-and-ai', './data-and-ai', './data-and-ai', './data-and-ai', './data-and-ai', './advanced-analytics', './advanced-analytics', './advanced-analytics', './data-and-ai', './data-and-ai', './cloud-and-architecture', './cloud-and-architecture', './cloud-and-architecture', './data-and-ai', './data-and-ai', './digital-tran

In [9]:
# Here is the system prompt for our LLM
link_system_prompt = "You are provided with a list of links found on a webpage.\
You must choose the links that would be the most relevant to include in a brochure about the company,\
such as links to an About page, or a Company page or a or Careers/Jobs pages or a Services page.\n"

link_system_prompt += "You should respond in JSON as in this example:"

link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"},
        {"type": "services page": "url": "https://another.full.url/services"}
    ]
}
"""

In [10]:
print(link_system_prompt)

You are provided with a list of links found on a webpage.You must choose the links that would be the most relevant to include in a brochure about the company,such as links to an About page, or a Company page or a or Careers/Jobs pages or a Services page.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"},
        {"type": "services page": "url": "https://another.full.url/services"}
    ]
}



In [11]:

def get_site_links(site):
    user_prompt = f"Here is the list of links on the website of {site.url} - "
    user_prompt = "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(site.links)
    return user_prompt
    

In [12]:
print(get_site_links(site))

please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
./#ai
./#ai
./#ai
./#ai
./story/mirageai-advantage#background
./#ai
./#ai
./#ai
./#ai
./story/mirageai-advantage#background
./#ai
./#ai
./#ai
./#ai
./story/mirageai-advantage#background
./#ai
./#ai
./#ai
./#ai
./story/mirageai-advantage#background
./
./mirage-ai
./about-us
./careers
./
./#ai
./story/mirageai-advantage#background
./#ai
./story/mirageai-advantage#background
./#ai
./story/mirageai-advantage#background
./#ai
./story/mirageai-advantage#background
./#ai
./#advanced
./#cloud
./#digital
./#financial
./#talent
./data-and-ai
./data-and-ai
./data-and-ai
./data-and-ai
./data-and-ai
./data-and-ai
./data-and-ai
./advanced-analytics
./advanced-analytics
./advanced-analytics
./data-and-ai
./data-and-ai
./cloud-and-architecture
./cloud-and-architecture
./cloud-an

In [13]:
# This method extracts the links present on the website we've scrapped
def get_links(url,headers):
    site = ScrapStaticWebsite(url,headers)
    response = openai.chat.completions.create(
        model = Model,
        messages = [
        {"role":"system","content":link_system_prompt},
        {"role":"user","content":get_site_links(site)}
        ],
        response_format = {"type":"json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)
    

In [14]:
get_links("https://www.jrvs.ca/",headers)

{'links': [{'type': 'about page', 'url': 'https://www.example.com/about-us'},
  {'type': 'careers page', 'url': 'https://www.example.com/careers'},
  {'type': 'company page', 'url': 'https://www.example.com/mirage-ai'},
  {'type': 'our impact page', 'url': 'https://www.example.com/our-impact'}]}

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [16]:
def get_all_details(url,headers):
    result = "Landing page:\n"
    result += ScrapStaticWebsite(url,headers).get_site_contents()
    links = get_links(url,headers)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += ScrapStaticWebsite(link["url"],headers).get_site_contents()
    return result

In [17]:
def get_brochure_user_prompt(company_name, url,headers):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url,headers)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("Jarvis","https://www.jrvs.ca",headers)

Found links: {'links': [{'type': 'about page', 'url': 'https://example.com/about-us'}, {'type': 'careers page', 'url': 'https://example.com/careers'}, {'type': 'services page', 'url': 'https://example.com/mirage-ai'}, {'type': 'impact page', 'url': 'https://example.com/our-impact'}, {'type': 'company story page', 'url': 'https://example.com/story/mirageai-advantage#background'}]}


"You are looking at a company called: Jarvis\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nJarvis Consulting Group | Innovate. Empower. Impact\nWebpage Contents:\nEMPOWERING AI, DATA & TECH\nINNOVATION FOR POSITIVE IMPACT\nOur Expertise\nMirageAI\nDATA MODERNIZATION WITHOUT COMPROMISING SECURITY\nLearn more\nEMPOWERING AI, DATA & TECH\nINNOVATION FOR POSITIVE IMPACT\nOur Expertise\nMirageAI\nDATA MODERNIZATION WITHOUT COMPROMISING SECURITY\nLearn more\nEMPOWERING AI, DATA & TECH\nINNOVATION FOR POSITIVE IMPACT\nOur Expertise\nMirageAI\nDATA MODERNIZATION WITHOUT COMPROMISING SECURITY\nLearn more\nEMPOWERING AI, DATA & TECH\nINNOVATION FOR POSITIVE IMPACT\nOur Expertise\nMirageAI\nDATA MODERNIZATION WITHOUT COMPROMISING SECURITY\nLearn more\nMirageAI\nServices\nCompany\nCareers\nSelect Language\nEnglish\nFrench (Canada)\nAt Jarvis,\nWE EMPOWER INNOVATION\nIN 

In [20]:
# save the content scraped into a text file
def save_generated_code(code):
    with open("brochure.txt", "w") as f:
        f.write(code)

# The function gpt_brochure has two main role:

* Get the website content, for each usefull links extract the information
* Pass this information to our LLM
* The LLM will extract the usefull information and store it
* This information will then be display in our gradio interface

In [21]:
def gpt_brochure(company_name,url):
    i = 0
    counts = []
    brochure_content = get_brochure_user_prompt(company_name,url,headers)
    stream = openai.chat.completions.create(
        model=Model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content":brochure_content}
          ],
        stream=True
    )
    response = ""
    #display_handle = display(Markdown(""), display_id=True)
    print("stream content:f{stream}")
    for chunk in stream:
        #i += 1
        #counts.append(f"Step {i+1} done")
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        #update_display(Markdown(response), display_id=display_handle.display_id)
        #gr.Markdown(f"Processing step {i+1}/{100}...\n\n{' '.join(counts)}")
        #print(i)
        yield response
    #save_generated_code(response)
    #return response

In [22]:
def claude_brochure(company_name,url):
    brochure_content = get_brochure_user_prompt(company_name,url,headers)
    result = claude.messages.stream(
     model = claude_model,
     messages = [{"role":"user","content":brochure_content}],
     system = system_prompt,
     max_tokens = 1000,
     temperature = 0.7
    )
    response = ""
    #display_handle = display(Markdown(""), display_id=True)
    with result as stream:
        for text in stream.text_stream:
            print(text)
            response+= text or ""
            response = response.replace("```","").replace("markdown", "")
            #update_display(Markdown(response), display_id=display_handle.display_id)
            yield response
        #save_generated_code(response)

# Chose the LLM to use for the brochure generation

In [23]:
def generate_brochure(company_name,url,model_name,progress=gr.Progress()):
    progress(0, desc="Starting...")
    if model_name == "GPT":
        print(model_name)
        result = gpt_brochure(company_name,url)
        print(result)
    elif model_name == "Claude":
        print(model_name)
        result = claude_brochure(company_name,url)
        print(result)
    else:
        raise ValueError("Unknown Error")
        
    for chunk in result:
        yield chunk
    

In [24]:
stream = gpt_brochure("Jarvis","https://www.jrvs.ca")

In [25]:
tlink_system_prompt = "You are a tool for translation. Your task is to translate a brochure from English into French.\
Each line of the brochure should be translate from english to french.\
Do not translate the email, phone number or any other link in the brochure."

In [26]:
def translate_brochure(user_prompt):
    
    stream = openai.chat.completions.create(
        model=Model,
        messages=[
            {"role": "system", "content": tlink_system_prompt},
            {"role": "user", "content": user_prompt}
          ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
    return response
    
    

In [27]:
with gr.Blocks() as ui:
    
    gr.Markdown("Generate a brochure and translate it")
    with gr.Row():
        generate = gr.Button("Generate")
        translate = gr.Button("Translate")
    with gr.Row():
        company_name = gr.Textbox(label="Company Name")
        website = gr.Textbox(label ="Website Url")
        models = gr.Dropdown(["GPT","Claude"],label="Select Model")
    with gr.Row():
        brochure = gr.Markdown(label="Brochure")
        translation = gr.Markdown(label="Translation")

    generate.click(generate_brochure,inputs=[company_name,website,models],outputs=[brochure])
    translate.click(translate_brochure,inputs=[brochure],outputs=[translation])

ui.launch()
            

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


GPT
<generator object gpt_brochure at 0x0000028874B6B640>
Found links: {'links': [{'type': 'about page', 'url': 'https://example.com/about-us'}, {'type': 'careers page', 'url': 'https://example.com/careers'}, {'type': 'services page', 'url': 'https://example.com/mirage-ai'}, {'type': 'impact page', 'url': 'https://example.com/our-impact'}]}
stream content:f{stream}


# Brochure du Jarvis Consulting Group

## Innover. Autonomiser. Impacter.

### Qui Nous Sommes
Chez **Jarvis Consulting Group**, nous permettons aux organisations de tirer parti de la puissance de l'**IA, des données et de la technologie** pour un impact positif. Nos conseillers et technologues experts s'attaquent à des défis complexes, en mettant à profit des technologies éprouvées et en proposant des solutions novatrices pour des résultats transformateurs. Nous croyons en la construction de relations durables avec nos clients grâce à une compréhension approfondie de leurs besoins uniques.

### Notre Expertise
- **MirageAI** : Solutions innovantes pour la **modernisation des données** sans compromettre la sécurité.
- **Données et IA** : Solutions sur mesure qui révolutionnent les pratiques commerciales grâce à une prise de décision basée sur les données.
- **Analytique Avancée** : Déverrouillez le potentiel de vos données avec des insights qui favorisent la croissance des affaires.
- **Cloud & Architecture** : Concevoir et gérer des infrastructures informatiques sécurisées et évolutives.
- **Transformation Numérique** : Redéfinir les opérations et l'engagement client pour adopter les innovations modernes.
- **Criminalité Financière & LBC** : Technologies avancées pour protéger contre la criminalité financière.

### Notre Approche
Chez Jarvis, nous transformons les concepts en réalité. Alors que les organisations se lancent dans leur parcours de transformation numérique, nous comprenons que la démocratisation des données est essentielle. Notre plateforme intégrée de données et d'IA garantit que les données deviennent exploitables et que les insights sont facilement accessibles. Nous visons à améliorer l'expérience client et à accroître la productivité, rendant les données disponibles pour les décideurs de manière innovante.

### Culture d'Entreprise
L'équipe de Jarvis prospère grâce à la collaboration, à l'innovation et à un engagement à avoir un impact positif. Nous nous consacrons à favoriser un environnement de travail inclusif qui nourrit la créativité et responsabilise chaque membre de l'équipe à contribuer à nos objectifs communs. Ensemble, nous ne sommes pas seulement une entreprise ; nous sommes une communauté engagée à faire avancer l'avenir de l'IA et des données.

### Carrières chez Jarvis
Nous sommes toujours à la recherche de talents qui partagent notre passion pour la technologie et l'innovation. Que vous soyez un professionnel expérimenté ou que vous débutiez votre carrière, Jarvis offre un lieu de travail dynamique où vous pouvez grandir et vous épanouir.

Rejoignez-nous alors que nous autonomisons l'innovation et entraînons des impacts positifs dans le monde de l'IA, des données et de la technologie.

### Connectez-vous avec Nous
Pour plus d'informations sur nos services, notre culture et nos opportunités de carrière, veuillez visiter notre site web [Jarvis Consulting Group](#).

---
**Votre avenir avec Jarvis vous attend.**  
Explorez le potentiel des données et de la technologie avec nous pour avoir un impact significatif dans votre organisation et au-delà.